In [1]:
import pandas as pd
import csv
import statsmodels.api as sm
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
file_url = '/Users/xzhou/github/project_archives/files_airbnb/la_airbnb/{}.csv'

listings_file = file_url.format('listings_details')
reviews_file = file_url.format('reviews_details')
calendar_file = file_url.format('calendar_details')

In [3]:
file_url = '/Users/xzhou/github/project_archives/files_airbnb/{}.csv'

sf_file = file_url.format('sf_airbnb/listings_details')
ok_file = file_url.format('ok_airbnb/listings_details')
sc_file = file_url.format('sc_airbnb/listings_details')
scz_file = file_url.format('scz_airbnb/listings_details')
pg_file = file_url.format('pg_airbnb/listings_details')
sd_file = file_url.format('sd_airbnb/listings_details')
la_file = file_url.format('la_airbnb/listings_details')

sea_file = file_url.format('sea_airbnb/listings_details')
bos_file = file_url.format('bos_airbnb/listings_details')

In [4]:
listings_sf = pd.read_csv(sf_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])
listings_ok = pd.read_csv(ok_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])
listings_sc = pd.read_csv(sc_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])
listings_scz = pd.read_csv(scz_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])
listings_sd = pd.read_csv(sd_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])
listings_la = pd.read_csv(la_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])

listings_sea = pd.read_csv(sea_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])
listings_bos = pd.read_csv(bos_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])

In [5]:
print (listings_sf.shape, listings_ok.shape, listings_sc.shape, listings_scz.shape, listings_sea.shape,listings_la.shape)

(6633, 96) (2898, 96) (5668, 96) (1570, 96) (3818, 92) (42337, 96)


In [6]:
frames = [listings_bos]

In [7]:
listings = pd.concat(frames)

In [8]:
listings.columns.values

array(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name',
       'summary', 'space', 'description', 'experiences_offered',
       'neighborhood_overview', 'notes', 'transit', 'access',
       'interaction', 'house_rules', 'thumbnail_url', 'medium_url',
       'picture_url', 'xl_picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode',
       'market', 'smart_location', 'country_code', 'country', 'latitude',
       'longitude', 'is_location_exact', 'property_type', 'room_type',
       'accommodates', 'bath

In [9]:
df = listings[["host_response_rate", "host_acceptance_rate", "host_is_superhost",
               "host_listings_count", "zipcode", "property_type","room_type", "accommodates", "bathrooms", "bedrooms", 
               "beds", "price", "number_of_reviews", "review_scores_rating", "cancellation_policy", 
               "reviews_per_month"]]

In [10]:
df.head()

,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,number_of_reviews,review_scores_rating,cancellation_policy,reviews_per_month
0,NaN,NaN,f,1,02131,House,Entire home/apt,4,1.5,2.0,3.0,$250.00,0,NaN,moderate,NaN
1,100%,100%,f,1,02131,Apartment,Private room,2,1.0,1.0,1.0,$65.00,36,94.0,moderate,1.30
2,100%,88%,t,1,02131,Apartment,Private room,2,1.0,1.0,1.0,$65.00,41,98.0,moderate,0.47
3,100%,50%,f,1,NaN,House,Private room,4,1.0,1.0,2.0,$75.00,1,100.0,moderate,1.00
4,100%,100%,t,1,02131,House,Private room,2,1.5,1.0,2.0,$79.00,29,99.0,flexible,2.25


In [11]:
# % of NaN values
(len(df)-df.count())/len(df)*100

host_response_rate      13.138075
host_acceptance_rate    13.138075
host_is_superhost        0.000000
host_listings_count      0.000000
zipcode                  1.059972
property_type            0.083682
room_type                0.000000
accommodates             0.000000
bathrooms                0.390516
bedrooms                 0.278940
beds                     0.251046
price                    0.000000
number_of_reviews        0.000000
review_scores_rating    22.677824
cancellation_policy      0.000000
reviews_per_month       21.087866
dtype: float64

In [12]:
df = df.drop(['host_acceptance_rate'], axis=1)

In [13]:
# drop NaN rows
df2=df.dropna(axis=0)

In [14]:
# % of rows droppped
(len(df)-len(df2))/len(df)*100

30.0976290097629

In [15]:
df2.shape

(2506, 15)

In [16]:
pd.options.mode.chained_assignment = None  # default='warn'

df2['host_response_rate'] = df2['host_response_rate'].astype(str)
df2['price'] = df2['price'].astype(str)

In [17]:
# clean data
pd.options.mode.chained_assignment = None  # default='warn'

df2['price'] = df2['price'].str.replace("[$, ]", "").astype("float")
df2['host_response_rate'] = df2['host_response_rate'].str.replace("%", "").astype("float")

In [18]:
df2['superhost']=np.where(df2['host_is_superhost']=='t',1,0)
del df2['host_is_superhost']

In [19]:
# select non-numeric variables and create dummies
non_num_vars = df2.select_dtypes(include=['object']).columns
df2[non_num_vars].head()

,zipcode,property_type,room_type,cancellation_policy
1,02131,Apartment,Private room,moderate
2,02131,Apartment,Private room,moderate
4,02131,House,Private room,flexible
5,02131,Condominium,Private room,flexible
6,02131,Apartment,Entire home/apt,strict


In [20]:
df4 = pd.get_dummies(df2, prefix=non_num_vars, columns=non_num_vars)
df4.shape

(2506, 70)

In [21]:
df4.shape

(2506, 70)

In [22]:
df4.groupby

<bound method NDFrame.groupby of       host_response_rate  host_listings_count  accommodates  bathrooms  \
1                  100.0                    1             2        1.0   
2                  100.0                    1             2        1.0   
4                  100.0                    1             2        1.5   
5                  100.0                    2             2        1.0   
6                   98.0                    5             3        1.0   
7                  100.0                    2             2        2.0   
8                  100.0                    1             2        1.0   
9                  100.0                    2             5        1.0   
10                  83.0                    1             2        1.0   
11                 100.0                    2             1        1.0   
12                 100.0                    2             4        1.0   
13                 100.0                    1             5        1.0   
14   

In [23]:
df4.columns

Index(['host_response_rate', 'host_listings_count', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'price', 'number_of_reviews',
       'review_scores_rating', 'reviews_per_month', 'superhost',
       'zipcode_02108', 'zipcode_02108 02111', 'zipcode_02109',
       'zipcode_02110', 'zipcode_02111', 'zipcode_02113', 'zipcode_02114',
       'zipcode_02115', 'zipcode_02116', 'zipcode_02118', 'zipcode_02119',
       'zipcode_02120', 'zipcode_02121', 'zipcode_02122', 'zipcode_02124',
       'zipcode_02125', 'zipcode_02126', 'zipcode_02127', 'zipcode_02128',
       'zipcode_02129', 'zipcode_02130', 'zipcode_02131', 'zipcode_02132',
       'zipcode_02134', 'zipcode_02134-1704', 'zipcode_02135', 'zipcode_02136',
       'zipcode_02138', 'zipcode_02139', 'zipcode_02141', 'zipcode_02142',
       'zipcode_02143', 'zipcode_02145', 'zipcode_02163', 'zipcode_02186',
       'zipcode_02210', 'zipcode_02215', 'zipcode_02445', 'zipcode_02446',
       'zipcode_02467', 'property_type_Apartment',
  

In [24]:
dummy_vars = pd.get_dummies(df2[non_num_vars])

In [25]:
dummy_vars.head()

,zipcode_02108,zipcode_02108 02111,zipcode_02109,zipcode_02110,zipcode_02111,zipcode_02113,zipcode_02114,zipcode_02115,zipcode_02116,zipcode_02118,...,property_type_Other,property_type_Townhouse,property_type_Villa,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_super_strict_30
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [26]:
# drop non-numeric variables from df2 and add the dummies
df3=df2.drop(non_num_vars,axis=1)
df3 = pd.merge(df3,dummy_vars, left_index=True, right_index=True)
df3.head()

,host_response_rate,host_listings_count,accommodates,bathrooms,bedrooms,beds,price,number_of_reviews,review_scores_rating,reviews_per_month,...,property_type_Other,property_type_Townhouse,property_type_Villa,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_super_strict_30
1,100.0,1,2,1.0,1.0,1.0,65.0,36,94.0,1.30,...,0,0,0,0,1,0,0,1,0,0
2,100.0,1,2,1.0,1.0,1.0,65.0,41,98.0,0.47,...,0,0,0,0,1,0,0,1,0,0
4,100.0,1,2,1.5,1.0,2.0,79.0,29,99.0,2.25,...,0,0,0,0,1,0,1,0,0,0
5,100.0,2,2,1.0,1.0,1.0,75.0,8,100.0,1.70,...,0,0,0,0,1,0,1,0,0,0
6,98.0,5,3,1.0,1.0,2.0,100.0,57,90.0,4.00,...,0,0,0,1,0,0,0,0,1,0


In [27]:
df3.shape

(2506, 70)

In [28]:
df

,host_response_rate,host_is_superhost,host_listings_count,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,number_of_reviews,review_scores_rating,cancellation_policy,reviews_per_month
0,NaN,f,1,02131,House,Entire home/apt,4,1.5,2.0,3.0,$250.00,0,NaN,moderate,NaN
1,100%,f,1,02131,Apartment,Private room,2,1.0,1.0,1.0,$65.00,36,94.0,moderate,1.30
2,100%,t,1,02131,Apartment,Private room,2,1.0,1.0,1.0,$65.00,41,98.0,moderate,0.47
3,100%,f,1,NaN,House,Private room,4,1.0,1.0,2.0,$75.00,1,100.0,moderate,1.00
4,100%,t,1,02131,House,Private room,2,1.5,1.0,2.0,$79.00,29,99.0,flexible,2.25
5,100%,t,2,02131,Condominium,Private room,2,1.0,1.0,1.0,$75.00,8,100.0,flexible,1.70
6,98%,f,5,02131,Apartment,Entire home/apt,3,1.0,1.0,2.0,$100.00,57,90.0,strict,4.00
7,100%,t,2,02131,House,Private room,2,2.0,1.0,1.0,$75.00,67,96.0,moderate,2.38
8,100%,t,1,02131,Condominium,Private room,2,1.0,1.0,2.0,$58.00,65,96.0,moderate,5.36
9,100%,t,2,02131,Apartment,Entire home/apt,5,1.0,2.0,2.0,$229.00,33,94.0,strict,1.01


In [29]:
# split into test and training data
np.random.seed(1)
indices = np.random.permutation(len(df4))
train_size = int(round(0.8*len(df4)))
test_size = len(df4)-train_size

y = df4['price']
x = df4.drop('price', axis =1)

x.train = x.iloc[indices[0:train_size]]
y.train = y.iloc[indices[0:train_size]]
x.test = x.iloc[indices[train_size+1:]]
y.test = y.iloc[indices[train_size+1:]]

x2 = x.train.as_matrix()
y2 = y.train.as_matrix()

/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  # Remove the CWD from sys.path while we load stuff.
/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  if sys.path[0] == '':
/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app
/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()


In [30]:
import statsmodels.api as sm
olsmod = sm.OLS(y2,x2)
olsres = olsmod.fit()
print(olsres.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.673
Model:                            OLS   Adj. R-squared:                  0.663
Method:                 Least Squares   F-statistic:                     65.64
Date:                Sat, 15 Sep 2018   Prob (F-statistic):               0.00
Time:                        22:23:09   Log-Likelihood:                -11192.
No. Observations:                2005   AIC:                         2.251e+04
Df Residuals:                    1943   BIC:                         2.285e+04
Df Model:                          61                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0998      0.142      0.704      0.4

In [31]:
# different method
from sklearn import linear_model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state=1)

In [32]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators=200, 
                               criterion='mse', 
                               random_state=3, 
                               n_jobs=-1)
forest.fit(X_train, y_train)
y_train_pred = forest.predict(X_train)
y_test_pred = forest.predict(X_test)

print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

MSE train: 608.638, test: 3646.415
R^2 train: 0.953, test: 0.646
